In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from utils import *

# First Dataset: "games_details.csv"

Have a look to the Datasets, one at a time. Firstly, "games_details.csv".

In [ ]:
games_details = pd.read_csv('../dataset/games_details.csv', low_memory=False)
games_details.head()

In [ ]:
# ratio_missing_values prints the percentage of missing values in the column
key1 = 'COMMENT'
ratio_missing_values_column(games_details, key1)

key2 = 'START_POSITION'
ratio_missing_values_column(games_details, key2)

'COMMENT': Since the number of valid values is very little wrt the size of the Dataset and there are not clear solutions to fill empty cells, we drop it.

'START_POSITION': same thing.

Moreover, we drop all the columns we think could lead to information leakage and also all columns that we think are not useful in the training model, such as the Nickname of the player. Our cleaned Dataset is the following

In [ ]:
Weird_rows = games_details['GAME_ID'] == 10500109
games_details = games_details[~Weird_rows]

columns_to_drop = ['MIN','COMMENT', 'PLAYER_NAME', 'NICKNAME', 'START_POSITION', 'COMMENT', 'TEAM_CITY', 'TEAM_ABBREVIATION', 'FGA', 'FG_PCT', 'FGM', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS']
games_details = games_details.drop(columns = columns_to_drop) # se non conta, si toglie. Altrimenti
# modificarla cambia il risultato
games_details = games_details.dropna()
games_details = games_details.reset_index(drop=True)

games_details.info()
games_details.head()

# Second Dataset: "games.csv"

Now, pass to the second Dataset, which is "games.csv".

In [ ]:
games = pd.read_csv('../dataset/games.csv')
games.head()

Now we add some features: bla bla bla

In [ ]:
for j in games['GAME_DATE_EST']:
    games['GAME_DATE_EST'] = games['GAME_DATE_EST'].replace(j,StringToDate(j))

games = games.sort_values(by='GAME_DATE_EST',ascending=False)
games = games.reset_index(drop=True)

In [ ]:
teams = pd.read_csv('../dataset/teams.csv')
teams.info()
teams.head()

# Third Dataset: "ranking.csv"
Aggiungere solo ultima partita della squadra avversaria e di quella del giocatore

In [ ]:
ranking = pd.read_csv('../dataset/ranking.csv')
ranking.info()
ranking = ranking.drop(columns=['LEAGUE_ID', 'RETURNTOPLAY']) # se soli zeri/NaN: rimosse
ranking.head()

# Fourth Dataset: "teams.csv"

In [ ]:
teams = teams.drop(columns=['LEAGUE_ID', 'MAX_YEAR', 'MIN_YEAR','ABBREVIATION','NICKNAME','YEARFOUNDED','CITY','ARENA','ARENACAPACITY','OWNER','GENERALMANAGER','HEADCOACH','DLEAGUEAFFILIATION'])

In [ ]:
# Computes 2D arrays for past games and winrates for each team

gamesTeamPlayed = [] #2D array of game_id's
gamesTeamDates = [] #2D array of distances from last game (when defined)
gamesTeamWinrates = [] #2D array of winrates in the past 3 matches (when defined)
for i in teams['TEAM_ID']:
    gamesHome = games['HOME_TEAM_ID'] == i
    gamesVisitor = games['VISITOR_TEAM_ID'] == i
    gamesTeamI = [x or y for x, y in zip(gamesHome,gamesVisitor)]
    tempGamesTeamPlayed = []
    for l in range(len(gamesTeamI)):
        if gamesTeamI[l]:
            tempGamesTeamPlayed.append(games['GAME_ID'][l])
    gamesTeamPlayed.append(tempGamesTeamPlayed)
    indices = []
    counter = 0
    for j in gamesTeamI:
        if j:
            indices.append(counter)
        counter = counter + 1
    gamesTeamDates.append([games['GAME_DATE_EST'][j] for j in indices])
    winrates = []
    for k in range(len(indices)-1,-1,-1):
        if len(indices)-1-k == 0:
            winrates.append(-1)
        elif len(indices)-1-k == 1:
            home = 1 if (games['HOME_TEAM_ID'][indices[k+1]] == i) else 0
            winrate = home*(games['HOME_TEAM_WINS'][indices[k+1]])+(1-home)*(1-games['HOME_TEAM_WINS'][indices[k+1]])
            winrates.append(winrate)
        elif len(indices)-1-k == 2:
            home = 1 if (games['HOME_TEAM_ID'][indices[k+1]] == i) else 0
            winrate = (home*(games['HOME_TEAM_WINS'][indices[k+1]])+(1-home)*(1-games['HOME_TEAM_WINS'][indices[k+1]]) + winrates[1])/2
            winrates.append(winrate)
        elif len(indices)-1-k >= 3:
            home = 1 if (games['HOME_TEAM_ID'][indices[k+1]] == i) else 0
            winrate = home*(games['HOME_TEAM_WINS'][indices[k+1]])+(1-home)*(1-games['HOME_TEAM_WINS'][indices[k+1]])
            home = 1 if (games['HOME_TEAM_ID'][indices[k+2]] == i) else 0
            winrate = winrate + home*(games['HOME_TEAM_WINS'][indices[k+2]])+(1-home)*(1-games['HOME_TEAM_WINS'][indices[k+2]])
            home = 1 if (games['HOME_TEAM_ID'][indices[k+3]] == i) else 0
            winrate = winrate + home*(games['HOME_TEAM_WINS'][indices[k+3]])+(1-home)*(1-games['HOME_TEAM_WINS'][indices[k+3]])
            winrates.append(winrate/3)
    winratesRev = []
    for h in range(len(winrates)-1,-1,-1):
        winratesRev.append(winrates[h])
    gamesTeamWinrates.append(winratesRev)
    

        
def DiffOppWin(game_id,team_id):
    indGameId = games.index[games['GAME_ID'] == game_id].tolist()[0]
    indTeamId = teams['TEAM_ID'].index[teams['TEAM_ID'] == team_id].tolist()[0]
    date = games['GAME_DATE_EST'][indGameId]
    dateId = gamesTeamDates[indTeamId].index(date)
    diff = -1
    if dateId == len(gamesTeamDates[indTeamId])-1:
        diff = -1
    else:
        diff = (date - gamesTeamDates[indTeamId][gamesTeamDates[indTeamId].index(date)+1])
    opposing = 0
    if games['HOME_TEAM_ID'][indGameId] == team_id:
        opposing = games['VISITOR_TEAM_ID'][indGameId]
    else:
        opposing = games['HOME_TEAM_ID'][indGameId]
    winrate = gamesTeamWinrates[indTeamId][gamesTeamPlayed[indTeamId].index(game_id)]
    return [diff, opposing, winrate]

def RefinedWinrate(game_id,home_id,visitor_id): #This is meant to edit games, not games_details
    indGameId = games.index[games['GAME_ID'] == game_id].tolist()[0]
    indTeamId = teams['TEAM_ID'].index[teams['TEAM_ID'] == home_id].tolist()[0]
    indOppTeamId = teams['TEAM_ID'].index[teams['TEAM_ID'] == visitor_id].tolist()[0]
    previousGames1 = [gamesTeamPlayed[indTeamId][j] for j in range(gamesTeamPlayed[indTeamId].index(indGameId),len(gamesTeamPlayed[indTeamId]))]
    previousGames2 = [gamesTeamPlayed[indOppTeamId][j] for j in range(gamesTeamPlayed[indOppTeamId].index(indGameId),len(gamesTeamPlayed[indOppTeamId]))]
    commonGames = [id for id in previousGames1 if id in previousGames2]
    if len(commonGames) == 0:
        return -1
    else:
        wins = 0
        for i in commonGames:
            wins = wins + games['HOME_TEAM_WINS'][i]
        wins = wins / len(commonGames)
        return wins

In [ ]:
# Adds a column with distance from the last game, winrate and opponent team for every entry in games_details (circa ?? minuti sul fisso, 20 sul portatile)

games_details['DATE_DIFF'] = ''
games_details['OPPOSING_TEAM'] = ''
games_details['WINRATE'] = ''
for i in range(len(games_details['DATE_DIFF'])):
    games_details.loc[i,'DATE_DIFF'] = DiffOppWin(games_details['GAME_ID'][i],games_details['TEAM_ID'][i])[0]
    games_details.loc[i,'OPPOSING_TEAM'] = DiffOppWin(games_details['GAME_ID'][i],games_details['TEAM_ID'][i])[1]
    games_details.loc[i,'WINRATE'] = DiffOppWin(games_details['GAME_ID'][i],games_details['TEAM_ID'][i])[2]

# Aggiungere qui le triplozze medie (usando il fatto che averageThreePointers è ordinato come game_details rispetto ad ogni giocatore)
# RICORDARSI DI TOGLIERE LE RIGHE CON -1

In [ ]:
games_details.head(10)

In [ ]:
print(games[games['GAME_STATUS_TEXT']=='Final'].equals(games)) # so no useful information, Final is the content of each cell
games = games.drop(columns=['GAME_STATUS_TEXT'])

games_details = complete_games_details(games_details, games)

games_details['TEAM_ID'].astype(str)
games_details['OPPOSING_TEAM_ID'].astype(str) # trasformare in stringhe nome così  da non lavorare
# con numeri enormi vicini ma usare l'encoder e pace

ratio_missing_values_df(games_details) # print percentage of rows in which there is at least one
# missing value
games_details = games_details.dropna() # drop all the rows with nans
games_details = games_details.reset_index(drop=True) # adjust the indexing

games_details.head()

In [ ]:
games_details = games_details.drop(columns=['OPPOSING_TEAM_ID'])
games_details.head(30)

In [ ]:
games_details.to_csv('../dataset/complete_dataset.csv', index=False)

Looking the Dataset we can notice that some arena capacity values are missing. We've decided to fill it searching the values on google:\
-Smoothie King Center: 17,805 seats;\
-Barclays Center: 17.732 seats;\
-Wells Fargo Center: 20,318 seats;\
-Talking Stick Resort Arena: 17,071 seats;

Moreover, Amway Center capcity seems to be wrong because its value is 0. So, we correct it:\
-Amway Center: 18,846 seats

In [ ]:
teams.loc[2, 'ARENACAPACITY'] = 17805.0
teams.loc[12, 'ARENACAPACITY'] = 17732.0
teams.loc[14, 'ARENACAPACITY'] = 18846.0
teams.loc[16, 'ARENACAPACITY'] = 20318.0
teams.loc[17, 'ARENACAPACITY'] = 17071.0

# Fifth Dataset: 'players.csv'

In [ ]:
players = pd.read_csv('../dataset/players.csv')
players.info()
players